In [1]:
import torch
from tqdm.notebook import tqdm
import pandas as pd
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

In [2]:
df = pd.read_csv('FinaldataNLP.csv')

In [3]:
df

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,hashtags,Topic_no
0,0,Destruction/Consequences,0.2297,"global, warming, park, stop, joshua, admin, na...","['global', 'wine', 'production', 'reach', 'new...",climatechange wine plastics,1
1,1,News/Media,0.2824,"threat, news, happening, temperature, day, fac...","['im', 'dismissing', 'crazy', 'conspiracy', 'i...",NaN,2
2,2,Belief/Sentiment,0.4131,"world, trump, head, president, study, way, sup...","['let', 'turn', 'thing', 'around']",ProtectWhatYouLove ActOnClimate Sustainability...,3
3,3,Arctic/Icecap,0.5074,"it, time, year, arctic, planet, weather, probl...","['new', 'approach', 'globalwarming', 'projecti...",NaN,4
4,4,ClimateChangeIsReal/FightClimateChange,0.7851,"climate, change, zinke, real, human, talk, wor...","['child', 'estimated', 'bear', 'burden', 'dise...",NaN,5
...,...,...,...,...,...,...,...
99995,328764,Energy/Emission/Carbon/,0.1880,"tree, energy, big, fuel, carbon, gas, get, rot...","['please', 'start', 'voting', 'candidate', 'ca...",NaN,11
99996,328768,Energy/Emission/Carbon/,0.2140,"tree, energy, big, fuel, carbon, gas, get, rot...","['year', 'projectpresident', 'donald', 'j', 't...",YEARSproject WarOnOurFuture,11
99997,328775,News/Media,0.2705,"threat, news, happening, temperature, day, fac...","['pollutionwatch', 'air', 'contamination', 'dr...",ClimateChange,2
99998,328777,xxForeign,0.2583,"scientist, security, trump, extreme, u, worse,...","['data', 'compelling', 'look', 'smaller', 'dat...",NaN,8


In [4]:
df['Dominant_Topic'].value_counts()

ClimateChangeIsReal/FightClimateChange    25378
Action/Efforts/Awareness                  19469
Destruction/Consequences                  11917
Belief/Sentiment                          10280
Arctic/Icecap                              7802
Energy/Emission/Carbon/                    7453
Politics/Policy/Law                        5027
PolarBear/Wildlife                         3742
xxForeign                                  3737
Paris Agreement                            2735
News/Media                                 2460
Name: Dominant_Topic, dtype: int64

In [5]:
possible_labels = df.Dominant_Topic.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'Destruction/Consequences': 0,
 'News/Media': 1,
 'Belief/Sentiment': 2,
 'Arctic/Icecap': 3,
 'ClimateChangeIsReal/FightClimateChange': 4,
 'Politics/Policy/Law': 5,
 'Action/Efforts/Awareness': 6,
 'xxForeign': 7,
 'PolarBear/Wildlife': 8,
 'Paris Agreement': 9,
 'Energy/Emission/Carbon/': 10}

In [6]:
df['label'] = df.Dominant_Topic.replace(label_dict)

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df.groupby(['Dominant_Topic', 'label', 'data_type']).count()

Document_No  \
Dominant_Topic                         label data_type                
Action/Efforts/Awareness               6     train            16549   
                                             val               2920   
Arctic/Icecap                          3     train             6632   
                                             val               1170   
Belief/Sentiment                       2     train             8738   
                                             val               1542   
ClimateChangeIsReal/FightClimateChange 4     train            21571   
                                             val               3807   
Destruction/Consequences               0     train            10129   
                                             val               1788   
Energy/Emission/Carbon/                10    train             6335   
                                             val               1118   
News/Media                             1     train             2091   
                                             val                369   
Paris Agreement                        9     train             2325   
                                             val                410   
PolarBear/Wildlife                     8     train             3181   
                                             val                561   
Politics/Policy/Law                    5     train             4273   
                                             val                754   
xxForeign                              7     train             3176   
                                             val                561   

                                                        Topic_Perc_Contrib  \
Dominant_Topic                         label data_type                       
Action/Efforts/Awareness               6     train                   16549   
                                             val                      2920   
Arctic/Icecap                          3     train                    6632   
                                             val                      1170   
Belief/Sentiment                       2     train                    8738   
                                             val                      1542   
ClimateChangeIsReal/FightClimateChange 4     train                   21571   
                                             val                      3807   
Destruction/Consequences               0     train                   10129   
                                             val                      1788   
Energy/Emission/Carbon/                10    train                    6335   
                                             val                      1118   
News/Media                             1     train                    2091   
                                             val                       369   
Paris Agreement                        9     train                    2325   
                                             val                       410   
PolarBear/Wildlife                     8     train                    3181   
                                             val                       561   
Politics/Policy/Law                    5     train                    4273   
                                             val                       754   
xxForeign                              7     train                    3176   
                                             val                       561   

                                                        Keywords   Text  \
Dominant_Topic                         label data_type                    
Action/Efforts/Awareness               6     train         16549  16549   
                                             val            2920   2920   
Arctic/Icecap                          3     train          6632   6632   
                                             val            1170   1170   
Belief/Sentiment            

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].Text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].Text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/govin/anaconda3/envs/transformers/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2136: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [9]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [10]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [11]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 2

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [12]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [13]:
import random
import numpy as np
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

cpu



Epoch 1
Training loss: 1.787939829548102
Validation loss: 1.567190859824419
F1 Score (Weighted): 0.4602538236563847



Epoch 2
Training loss: 1.4202015810188013
Validation loss: 1.3975628977119923
F1 Score (Weighted): 0.5290076500645929

